In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from random import random

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [27]:
import keras
keras.__version__

'2.5.0'

# Data Split

In [28]:
import random  
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4068), (6182,), (1545, 10, 4068), (1545,))

# Revised KerasClassifier
- https://github.com/veniversum/keras/blob/9a401eb2e184fda7238a6259c1b8b02c645e4e9c/keras/wrappers/scikit_learn.py

In [29]:
import sklearn
import sklearn.utils

In [30]:
"""Wrapper for using the Scikit-Learn API with Keras models.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import copy
import types

import numpy as np
import sklearn
from tensorflow import keras
from keras.utils.np_utils import to_categorical
from keras.utils.generic_utils import has_arg
from keras.models import Sequential
# from keras.layers import BaseWrapper

class BaseWrapper(object):
    """Base class for the Keras scikit-learn wrapper.

    Warning: This class should not be used directly.
    Use descendant classes instead.

    # Arguments
        build_fn: callable function or class instance
        **sk_params: model parameters & fitting parameters

    The `build_fn` should construct, compile and return a Keras model, which
    will then be used to fit/predict. One of the following
    three values could be passed to `build_fn`:
    1. A function
    2. An instance of a class that implements the `__call__` method
    3. None. This means you implement a class that inherits from either
    `KerasClassifier` or `KerasRegressor`. The `__call__` method of the
    present class will then be treated as the default `build_fn`.

    `sk_params` takes both model parameters and fitting parameters. Legal model
    parameters are the arguments of `build_fn`. Note that like all other
    estimators in scikit-learn, `build_fn` should provide default values for
    its arguments, so that you could create the estimator without passing any
    values to `sk_params`.

    `sk_params` could also accept parameters for calling `fit`, `predict`,
    `predict_proba`, and `score` methods (e.g., `epochs`, `batch_size`).
    fitting (predicting) parameters are selected in the following order:

    1. Values passed to the dictionary arguments of
    `fit`, `predict`, `predict_proba`, and `score` methods
    2. Values passed to `sk_params`
    3. The default values of the `keras.models.Sequential`
    `fit`, `predict`, `predict_proba` and `score` methods

    When using scikit-learn's `grid_search` API, legal tunable parameters are
    those you could pass to `sk_params`, including fitting parameters.
    In other words, you could use `grid_search` to search for the best
    `batch_size` or `epochs` as well as the model parameters.
    """

    def __init__(self, build_fn=None, **sk_params):
        self.build_fn = build_fn
        self.sk_params = sk_params
        self.check_params(sk_params)

    def check_params(self, params):
        """Checks for user typos in `params`.

        # Arguments
            params: dictionary; the parameters to be checked

        # Raises
            ValueError: if any member of `params` is not a valid argument.
        """
        legal_params_fns = [Sequential.fit, Sequential.predict,
                            Sequential.predict_classes, Sequential.evaluate]
        if self.build_fn is None:
            legal_params_fns.append(self.__call__)
        elif (not isinstance(self.build_fn, types.FunctionType) and
              not isinstance(self.build_fn, types.MethodType)):
            legal_params_fns.append(self.build_fn.__call__)
        else:
            legal_params_fns.append(self.build_fn)

        for params_name in params:
            for fn in legal_params_fns:
                if has_arg(fn, params_name):
                    break
            else:
                if params_name != 'nb_epoch':
                    raise ValueError(
                        '{} is not a legal parameter'.format(params_name))

    def get_params(self, **params):
        """Gets parameters for this estimator.

        # Arguments
            **params: ignored (exists for API compatibility).

        # Returns
            Dictionary of parameter names mapped to their values.
        """
        res = copy.deepcopy(self.sk_params)
        res.update({'build_fn': self.build_fn})
        return res

    def set_params(self, **params):
        """Sets the parameters of this estimator.

        # Arguments
            **params: Dictionary of parameter names mapped to their values.

        # Returns
            self
        """
        self.check_params(params)
        self.sk_params.update(params)
        return self

    def fit(self, x, y, **kwargs):
        """Constructs a new model with `build_fn` & fit the model to `(x, y)`.

        # Arguments
            x : array-like, shape `(n_samples, n_features)`
                Training samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y : array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.fit`

        # Returns
            history : object
                details about the training history at each epoch.
        """
        if self.build_fn is None:
            self.model = self.__call__(**self.filter_sk_params(self.__call__))
        elif (not isinstance(self.build_fn, types.FunctionType) and
              not isinstance(self.build_fn, types.MethodType)):
            self.model = self.build_fn(
                **self.filter_sk_params(self.build_fn.__call__))
        else:
            self.model = self.build_fn(**self.filter_sk_params(self.build_fn))

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)

        fit_args = copy.deepcopy(self.filter_sk_params(Sequential.fit))
        fit_args.update(kwargs)

        history = self.model.fit(x, y, **fit_args)

        return history

    def filter_sk_params(self, fn, override=None):
        """Filters `sk_params` and returns those in `fn`'s arguments.

        # Arguments
            fn : arbitrary function
            override: dictionary, values to override `sk_params`

        # Returns
            res : dictionary containing variables
                in both `sk_params` and `fn`'s arguments.
        """
        override = override or {}
        res = {}
        for name, value in self.sk_params.items():
            if has_arg(fn, name):
                res.update({name: value})
        res.update(override)
        return res


class KerasClassifier(BaseWrapper):
    """Implementation of the scikit-learn classifier API for Keras.
    """

    def fit(self, x, y, sample_weight=None, **kwargs):
        """Constructs a new model with `build_fn` & fit the model to `(x, y)`.

        # Arguments
            x : array-like, shape `(n_samples, n_features)`
                Training samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y : array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.fit`

        # Returns
            history : object
                details about the training history at each epoch.

        # Raises
            ValueError: In case of invalid shape for `y` argument.
        """
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        if sample_weight is not None:
            
            # 1. new sample weight로 update, 2. 범위 지정 by tuple
            range_list = []
            update = 0
            for i in sample_weight:
                range_list.append((update, update + i/sum(sample_weight)))
                update += i/sum(sample_weight)

            # 3. 0~1 사이 값 len(sample_weight)개 random 추출
            randoms = [random.random() for _ in range(len(sample_weight))]
            
            x0 = x.copy()
            y0 = y.copy()
            for i in range(len(randoms)):
                for j in range(len(range_list)):
                    if range_list[j][0] < randoms[i] < range_list[j][1]:
                        x0[i] = x[j]
                        y0[i] = y[j]
                        break # for문 끊어서 시간 단축하기 위해.
            
            kwargs['sample_weight'] = sample_weight # 새로운 x, y를 만드는 과정이 현재 없다.
        return super(KerasClassifier, self).fit(x0, y0, **kwargs)

    def predict(self, x, **kwargs):
        """Returns the class predictions for the given test data.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            **kwargs: dictionary arguments
                Legal arguments are the arguments
                of `Sequential.predict_classes`.

        # Returns
            preds: array-like, shape `(n_samples,)`
                Class predictions.
        """
        kwargs = self.filter_sk_params(Sequential.predict_classes, kwargs)
        classes = self.model.predict_classes(x, **kwargs)
        return self.classes_[classes]

    def predict_proba(self, x, **kwargs):
        """Returns class probability estimates for the given test data.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            **kwargs: dictionary arguments
                Legal arguments are the arguments
                of `Sequential.predict_classes`.

        # Returns
            proba: array-like, shape `(n_samples, n_outputs)`
                Class probability estimates.
                In the case of binary classification,
                to match the scikit-learn API,
                will return an array of shape `(n_samples, 2)`
                (instead of `(n_sample, 1)` as in Keras).
        """
        kwargs = self.filter_sk_params(Sequential.predict_proba, kwargs)
        probs = self.model.predict_proba(x, **kwargs)

        # check if binary classification
        if probs.shape[1] == 1:
            # first column is probability of class 0 and second is of class 1
            probs = np.hstack([1 - probs, probs])
        return probs

    def score(self, x, y, **kwargs):
        """Returns the mean accuracy on the given test data and labels.

        # Arguments
            x: array-like, shape `(n_samples, n_features)`
                Test samples where `n_samples` is the number of samples
                and `n_features` is the number of features.
            y: array-like, shape `(n_samples,)` or `(n_samples, n_outputs)`
                True labels for `x`.
            **kwargs: dictionary arguments
                Legal arguments are the arguments of `Sequential.evaluate`.

        # Returns
            score: float
                Mean accuracy of predictions on `x` wrt. `y`.

        # Raises
            ValueError: If the underlying model isn't configured to
                compute accuracy. You should pass `metrics=["accuracy"]` to
                the `.compile()` method of the model.
        """
        y = np.searchsorted(self.classes_, y)
        kwargs = self.filter_sk_params(Sequential.evaluate, kwargs)

        loss_name = self.model.loss
        if hasattr(loss_name, '__name__'):
            loss_name = loss_name.__name__
        if loss_name == 'categorical_crossentropy' and len(y.shape) != 2:
            y = to_categorical(y)

        outputs = self.model.evaluate(x, y, **kwargs)
        if not isinstance(outputs, list):
            outputs = [outputs]
        for name, output in zip(self.model.metrics_names, outputs):
            if name == 'acc':
                return output
        raise ValueError('The model is not configured to compute accuracy. '
                         'You should pass `metrics=["accuracy"]` to '
                         'the `model.compile()` method.')

# Apply AdaboostClassifier
## adaboost1, get_model

In [31]:
seed_num = 42

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier

from sklearn import metrics 
from tensorflow import keras
# from keras.wrappers.scikit_learn import KerasClassifier
print(tf.__version__)
print(keras.__version__)


def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
    return lstm

2.5.0
2.5.0


## single LSTM
정확도 확인을 위해 하는 것.

In [32]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_acc', patience=10, verbose=1, restore_best_weights=False)

model = get_model()
model.fit(X_train,y_train, validation_split = 0.2, epochs=100, batch_size=64, callbacks=[early_stop])

preds = model.predict(X_test)
preds[preds>0.5] = 1
preds[preds<=0.5] = 0

precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
roc_auc = roc_auc_score(y_test, preds)
acc = accuracy_score(y_test, preds)

print(f' accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

Epoch 1/100
78/78 [==============================] - 9s 68ms/step - loss: 0.6776 - acc: 0.5893 - val_loss: 0.6665 - val_acc: 0.6112
Epoch 2/100
78/78 [==============================] - 5s 62ms/step - loss: 0.6369 - acc: 0.6362 - val_loss: 0.5516 - val_acc: 0.7510
Epoch 3/100
78/78 [==============================] - 5s 63ms/step - loss: 0.5021 - acc: 0.7749 - val_loss: 0.4944 - val_acc: 0.7761
Epoch 4/100
78/78 [==============================] - 5s 62ms/step - loss: 0.4191 - acc: 0.8257 - val_loss: 0.5074 - val_acc: 0.7736
Epoch 5/100
78/78 [==============================] - 5s 63ms/step - loss: 0.3688 - acc: 0.8615 - val_loss: 0.5332 - val_acc: 0.7631
Epoch 6/100
78/78 [==============================] - 5s 63ms/step - loss: 0.3240 - acc: 0.8837 - val_loss: 0.5660 - val_acc: 0.7518
Epoch 7/100
78/78 [==============================] - 5s 64ms/step - loss: 0.2932 - acc: 0.8993 - val_loss: 0.6138 - val_acc: 0.7437
Epoch 8/100
78/78 [==============================] - 5s 63ms/step - loss: 0.

## Adaboost

In [34]:
# adaboost
lstm_Predictors = KerasClassifier(build_fn=lambda:get_model(), epochs=200, batch_size=1024) # epoch은 150이상, batch_size는 8의 배수로 hp 조정
lstm_Predictors._estimator_type="classifier"
final_model = AdaBoostClassifier(lstm_Predictors, n_estimators=5, random_state=42, learning_rate=1.2) # n_estimator 조정

In [35]:
%%time
with tf.device('/device:GPU:0'):
    final_model.fit(X_train,y_train, sample_weight=np.ones(X_train.shape[0])/X_train.shape[0])

Epoch 1/200
7/7 [==============================] - 4s 77ms/step - loss: 1.3127e-04 - acc: 0.3908
Epoch 2/200
7/7 [==============================] - 1s 82ms/step - loss: 1.2326e-04 - acc: 0.4057
Epoch 3/200
7/7 [==============================] - 1s 83ms/step - loss: 1.1704e-04 - acc: 0.4427
Epoch 4/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1245e-04 - acc: 0.5233
Epoch 5/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1045e-04 - acc: 0.5644
Epoch 6/200
7/7 [==============================] - 1s 81ms/step - loss: 1.0954e-04 - acc: 0.5893
Epoch 7/200
7/7 [==============================] - 1s 81ms/step - loss: 1.0977e-04 - acc: 0.5925
Epoch 8/200
7/7 [==============================] - 1s 81ms/step - loss: 1.0945e-04 - acc: 0.5988
Epoch 9/200
7/7 [==============================] - 1s 81ms/step - loss: 1.0941e-04 - acc: 0.6013
Epoch 10/200
7/7 [==============================] - 1s 80ms/step - loss: 1.0954e-04 - acc: 0.5964
Epoch 11/200
7/7 [===========

7/7 [==============================] - 1s 80ms/step - loss: 1.0872e-04 - acc: 0.6105
Epoch 79/200
7/7 [==============================] - 1s 80ms/step - loss: 1.0877e-04 - acc: 0.6077
Epoch 80/200
7/7 [==============================] - 1s 81ms/step - loss: 1.0868e-04 - acc: 0.6097
Epoch 81/200
7/7 [==============================] - 1s 82ms/step - loss: 1.0850e-04 - acc: 0.6102
Epoch 82/200
7/7 [==============================] - 1s 82ms/step - loss: 1.0887e-04 - acc: 0.6111
Epoch 83/200
7/7 [==============================] - 1s 81ms/step - loss: 1.0856e-04 - acc: 0.6106
Epoch 84/200
7/7 [==============================] - 1s 83ms/step - loss: 1.0839e-04 - acc: 0.6108
Epoch 85/200
7/7 [==============================] - 1s 80ms/step - loss: 1.0882e-04 - acc: 0.6123
Epoch 86/200
7/7 [==============================] - 1s 80ms/step - loss: 1.0860e-04 - acc: 0.6108
Epoch 87/200
7/7 [==============================] - 1s 82ms/step - loss: 1.0851e-04 - acc: 0.6113
Epoch 88/200
7/7 [===============

7/7 [==============================] - 1s 83ms/step - loss: 1.0807e-04 - acc: 0.6119
Epoch 162/200
7/7 [==============================] - 1s 79ms/step - loss: 1.0846e-04 - acc: 0.6118
Epoch 163/200
7/7 [==============================] - 1s 82ms/step - loss: 1.0837e-04 - acc: 0.6119
Epoch 164/200
7/7 [==============================] - 1s 83ms/step - loss: 1.0844e-04 - acc: 0.6113
Epoch 165/200
7/7 [==============================] - 1s 78ms/step - loss: 1.0817e-04 - acc: 0.6118
Epoch 166/200
7/7 [==============================] - 1s 81ms/step - loss: 1.0838e-04 - acc: 0.6118
Epoch 167/200
7/7 [==============================] - 1s 81ms/step - loss: 1.0828e-04 - acc: 0.6118
Epoch 168/200
7/7 [==============================] - 1s 82ms/step - loss: 1.0837e-04 - acc: 0.6119
Epoch 169/200
7/7 [==============================] - 1s 77ms/step - loss: 1.0840e-04 - acc: 0.6115
Epoch 170/200
7/7 [==============================] - 1s 80ms/step - loss: 1.0838e-04 - acc: 0.6118
Epoch 171/200
7/7 [=====

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Epoch 1/200
7/7 [==============================] - 4s 80ms/step - loss: 1.1574e-04 - acc: 0.4833
Epoch 2/200
7/7 [==============================] - 1s 80ms/step - loss: 1.1382e-04 - acc: 0.5002
Epoch 3/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1352e-04 - acc: 0.4997
Epoch 4/200
7/7 [==============================] - 1s 78ms/step - loss: 1.1276e-04 - acc: 0.5102
Epoch 5/200
7/7 [==============================] - 1s 84ms/step - loss: 1.1312e-04 - acc: 0.5018
Epoch 6/200
7/7 [==============================] - 1s 78ms/step - loss: 1.1306e-04 - acc: 0.5155
Epoch 7/200
7/7 [==============================] - 1s 78ms/step - loss: 1.1319e-04 - acc: 0.5036
Epoch 8/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1295e-04 - acc: 0.5008
Epoch 9/200
7/7 [==============================] - 1s 77ms/step - loss: 1.1289e-04 - acc: 0.5019
Epoch 10/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1326e-04 - acc: 0.5027
Epoch 11/200
7/7 [===========

...

Epoch 1/200
7/7 [==============================] - 4s 81ms/step - loss: 1.1765e-04 - acc: 0.5070
Epoch 2/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1551e-04 - acc: 0.5018
Epoch 3/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1399e-04 - acc: 0.5068
Epoch 4/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1377e-04 - acc: 0.4998
Epoch 5/200
7/7 [==============================] - 1s 80ms/step - loss: 1.1333e-04 - acc: 0.5057
Epoch 6/200
7/7 [==============================] - 1s 79ms/step - loss: 1.1384e-04 - acc: 0.4956
Epoch 7/200
7/7 [==============================] - 1s 79ms/step - loss: 1.1382e-04 - acc: 0.5005
Epoch 8/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1345e-04 - acc: 0.4956
Epoch 9/200
7/7 [==============================] - 1s 84ms/step - loss: 1.1357e-04 - acc: 0.4990
Epoch 10/200
7/7 [==============================] - 1s 85ms/step - loss: 1.1335e-04 - acc: 0.5006
Epoch 11/200
7/7 [===========

7/7 [==============================] - 1s 81ms/step - loss: 1.1253e-04 - acc: 0.5021
Epoch 82/200
7/7 [==============================] - 1s 83ms/step - loss: 1.1274e-04 - acc: 0.4829
Epoch 83/200
7/7 [==============================] - 1s 83ms/step - loss: 1.1253e-04 - acc: 0.5061
Epoch 84/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1254e-04 - acc: 0.4976
Epoch 85/200
7/7 [==============================] - 1s 80ms/step - loss: 1.1282e-04 - acc: 0.4859
Epoch 86/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1270e-04 - acc: 0.4968
Epoch 87/200
7/7 [==============================] - 1s 84ms/step - loss: 1.1246e-04 - acc: 0.5026
Epoch 88/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1244e-04 - acc: 0.5015
Epoch 89/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1264e-04 - acc: 0.4939
Epoch 90/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1262e-04 - acc: 0.4943
Epoch 91/200
7/7 [===============

7/7 [==============================] - 1s 80ms/step - loss: 1.1234e-04 - acc: 0.5031
Epoch 165/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1235e-04 - acc: 0.5021
Epoch 166/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1226e-04 - acc: 0.5034
Epoch 167/200
7/7 [==============================] - 1s 85ms/step - loss: 1.1217e-04 - acc: 0.5044
Epoch 168/200
7/7 [==============================] - 1s 80ms/step - loss: 1.1213e-04 - acc: 0.5053
Epoch 169/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1225e-04 - acc: 0.4992
Epoch 170/200
7/7 [==============================] - 1s 79ms/step - loss: 1.1231e-04 - acc: 0.4994
Epoch 171/200
7/7 [==============================] - 1s 79ms/step - loss: 1.1234e-04 - acc: 0.4963
Epoch 172/200
7/7 [==============================] - 1s 78ms/step - loss: 1.1212e-04 - acc: 0.5126
Epoch 173/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1222e-04 - acc: 0.5089
Epoch 174/200
7/7 [=====

7/7 [==============================] - 1s 82ms/step - loss: 1.1239e-04 - acc: 0.5073
Epoch 41/200
7/7 [==============================] - 1s 80ms/step - loss: 1.1285e-04 - acc: 0.4974
Epoch 42/200
7/7 [==============================] - 1s 80ms/step - loss: 1.1320e-04 - acc: 0.4864
Epoch 43/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1279e-04 - acc: 0.5008
Epoch 44/200
7/7 [==============================] - 1s 77ms/step - loss: 1.1262e-04 - acc: 0.4995
Epoch 45/200
7/7 [==============================] - 1s 79ms/step - loss: 1.1273e-04 - acc: 0.5008
Epoch 46/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1274e-04 - acc: 0.5013
Epoch 47/200
7/7 [==============================] - 1s 79ms/step - loss: 1.1304e-04 - acc: 0.4948
Epoch 48/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1272e-04 - acc: 0.5024
Epoch 49/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1284e-04 - acc: 0.4939
Epoch 50/200
7/7 [===============

7/7 [==============================] - 1s 79ms/step - loss: 1.1244e-04 - acc: 0.5016
Epoch 124/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1231e-04 - acc: 0.5036
Epoch 125/200
7/7 [==============================] - 1s 81ms/step - loss: 1.1226e-04 - acc: 0.4960
Epoch 126/200
7/7 [==============================] - 1s 79ms/step - loss: 1.1225e-04 - acc: 0.5000
Epoch 127/200
7/7 [==============================] - 1s 80ms/step - loss: 1.1232e-04 - acc: 0.5021
Epoch 128/200
7/7 [==============================] - 1s 79ms/step - loss: 1.1237e-04 - acc: 0.4968
Epoch 129/200
7/7 [==============================] - 1s 80ms/step - loss: 1.1230e-04 - acc: 0.5074
Epoch 130/200
7/7 [==============================] - 1s 78ms/step - loss: 1.1235e-04 - acc: 0.5018
Epoch 131/200
7/7 [==============================] - 1s 80ms/step - loss: 1.1239e-04 - acc: 0.5044
Epoch 132/200
7/7 [==============================] - 1s 82ms/step - loss: 1.1244e-04 - acc: 0.4969
Epoch 133/200
7/7 [=====

In [36]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
preds = final_model.predict(X_test)

precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
roc_auc = roc_auc_score(y_test, preds)
acc = accuracy_score(y_test, preds)

print(f' accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


 accuracy : 0.6006472491909385, precision : 0.6006472491909385, recall : 1.0, f1 : 0.7505054589567327, roc_auc : 0.5
